In [29]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation
from keras.utils import to_categorical
from keras.optimizers import Adadelta
from keras import backend as k
#Set to mini batch
batch_size = 128
#the number of classes we want to classify 0 --- 9
num_classes =10
#the size of the image in the mnist dataset
img_rows,img_columns = 28 ,28
#load the data to a trainning and testing
(x_train,y_train),(x_test,y_test) = mnist.load_data()
#changing the shapes to fit as we are using tensorflow as a backend
x_train = x_train.reshape(x_train.shape[0],img_rows,img_columns,1)
x_test = x_test.reshape(x_test.shape[0],img_rows,img_columns,1)
input_shape = (img_rows,img_columns,1)

#change the type for calcuations
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

#normilize the values
x_train /=255
x_test /=255

#convert the class vector to a binary matrix 
#the values a represnted as 0 0 0 0 0 0 0 0 0 0 
# and there will be one on the correct class
y_train = to_categorical(y_train,num_classes)
y_test = to_categorical(y_test,num_classes)

In [30]:
#create the model
model =  Sequential()
#add a conv2d layer with 32 fiters which will give us 26 26 32 the 32 is the number of channels 
model.add(Conv2D(32,(3,3),activation = 'relu',input_shape= input_shape))
# will give a 24 24 64 channels the image is decrising becuase we are not padding
model.add(Conv2D(64,(3,3),activation = 'relu'))
#to reduce the size of the image and just keep focusing on the most important features
model.add(MaxPooling2D(pool_size = (2,2)))

#now we need to flattern the image for use in dense layers
model.add(Flatten())
#the number of nurons is based on the project the this model is from
model.add(Dense(128,activation='relu'))
#add a drop out layer so we do not hit overfitting
#it will disable hlaf of the nurons in on every input
model.add(Dropout(0.5))
#the last dense layer for the class classifying
model.add(Dense(num_classes))
#add the softmax activation to get the values between 0 -- 1
model.add(Activation('softmax'))
#setup the model for training
model.compile(loss = 'categorical_crossentropy',
              optimizer = Adadelta(),
              #We can specify more metrics that we want to monitor
              metrics=['accuracy'])



In [31]:
#the number of params for a conv2d layer is (filterSize)*No.Filters + No.Filters
#                                            (3 * 3) * 32 + 32
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                1290      
__________

In [38]:
#train the model
model.fit(x = x_train,y = y_train,
          batch_size = batch_size,
          epochs = 15,
        #this will show is a progress bar put it to 0 so it can remain silent
          verbose =1,
          validation_data = (x_test,y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 345s 6ms/step - loss: 0.0852 - acc: 0.9742 - val_loss: 0.0396 - val_acc: 0.9865
Epoch 2/15
60000/60000 [==============================] - 429s 7ms/step - loss: 0.0615 - acc: 0.9819 - val_loss: 0.0337 - val_acc: 0.9881
Epoch 3/15
60000/60000 [==============================] - 428s 7ms/step - loss: 0.0509 - acc: 0.9851 - val_loss: 0.0324 - val_acc: 0.9888
Epoch 4/15
60000/60000 [==============================] - 388s 6ms/step - loss: 0.0424 - acc: 0.9868 - val_loss: 0.0307 - val_acc: 0.9897
Epoch 5/15
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0380 - acc: 0.9884 - val_loss: 0.0311 - val_acc: 0.9900
Epoch 6/15
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0315 - acc: 0.9908 - val_loss: 0.0292 - val_acc: 0.9903
Epoch 7/15
60000/60000 [==============================] - 132s 2ms/step - loss: 0.0273 - acc: 0.9912 - val_loss: 0.0278 - 

In [ ]:
#evaluate the model
#for more information run model.metrices_names == > ['loss' , 'acc']
scores = model.evaluate(x = x_test,y=y_test,batch_size = 32,verbose = 1)

print("The scroes : " + scores)